## Processing

In [1]:
# Set Project Root
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

# import my method from the source code
# from features.build_features import remove_invalid_data
# from features.build_features import awesome_function


import os
import sys

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly as plty

import missingno as msno

from wordcloud import WordCloud
from pandas_profiling import ProfileReport

import html
import re

# Textacy
import textacy.preprocessing as tprep # Preprocesing of accents/normalization
from textacy.preprocessing.resources import RE_URL

import nltk
from nltk.tokenize import RegexpTokenizer

# Set to Reload all custom packages
%load_ext autoreload
%autoreload 2


# Custom developed tools
import nlp_tools
import constants

nltk.download('stopwords')
nltk.download('wordnet')


g:\My Drive\Code\springboard_proj_nlp_nytimes


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load up data

In [2]:
# Load Data
df_test = pd.read_pickle('..//data//processed//df_test_sample.pkl')
df_train = pd.read_pickle('..//data//processed//df_train_sample.pkl')
df_comments = pd.read_pickle('..//data//processed//df_comments_sample.pkl')
df_articles = pd.read_pickle('..//data//processed//df_articles_sample.pkl')

# # Load Data
# df_test = pd.read_pickle('..//data//raw//df_test.pkl')
# df_train = pd.read_pickle('..//data//raw//df_train.pkl')
# df_comments = pd.read_pickle('..//data//raw//df_comments.pkl')
# df_articles = pd.read_pickle('..//data//raw//df_articles.pkl')

# EDA

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2558 entries, 8527 to 10328
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   newsdesk    2558 non-null   object
 1   section     2558 non-null   object
 2   subsection  883 non-null    object
 3   material    2558 non-null   object
 4   headline    2558 non-null   object
 5   abstract    2557 non-null   object
 6   keywords    2558 non-null   object
 7   word_count  2558 non-null   int64 
 8   pub_date    2558 non-null   object
 9   is_popular  2558 non-null   int64 
 10  n_comments  2558 non-null   int64 
 11  uniqueID    2558 non-null   object
dtypes: int64(3), object(9)
memory usage: 259.8+ KB


In [4]:
df_train.head()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,is_popular,n_comments,uniqueID
8527,Foreign,World,Europe,News,France Brings 10 Children of French Jihadists ...,About 270 children of French citizens remain s...,"['France', 'Islamic State in Iraq and Syria (I...",1131,2020-06-22 21:20:53+00:00,0,6,nyt://article/f1e6b417-f8ca-597d-9be9-db2ca917...
322,Politics,U.S.,Politics,News,How Elizabeth Warren Is Being Squeezed by 2 De...,Ms. Warren is figuring out what candidates lik...,"['Presidential Election of 2020', 'Warren, Eli...",1442,2020-01-08 19:12:09+00:00,1,397,nyt://article/88600e52-644c-56c3-805f-af931533...
6565,Styles,Style,NaN,News,They Fled the Coronavirus. Now They’re in Scen...,Some feel sheepish about their choice to retre...,"['Travel and Vacations', 'Coronavirus (2019-nC...",1989,2020-05-11 09:00:20+00:00,1,362,nyt://article/fd7d10a8-9bc0-55b2-8829-0ac8b275...
5655,Learning,The Learning Network,NaN,News,How Do You Greet Your Friends and Family?,Do you hug friends or use handshakes? How have...,[],648,2020-04-23 12:19:17+00:00,1,168,nyt://article/2554b892-d3b3-505e-8cc8-d1ddbf5a...
10337,Foreign,World,Australia,News,"What Lockdown 2.0 Looks Like: Harsher Rules, D...","Melbourne, Australia’s second-largest city, is...","['Quarantines', 'Coronavirus (2019-nCoV)', 'Au...",1505,2020-08-04 09:55:38+00:00,1,294,nyt://article/ba22d655-75b2-57ca-8822-3b071e7c...


In [5]:
df_train.groupby('pub_date').agg({'word_count':'mean'})

,word_count
pub_date,
2020-01-01 03:00:10+00:00,931.0
2020-01-01 05:00:12+00:00,0.0
2020-01-01 10:00:01+00:00,153.0
2020-01-01 10:01:24+00:00,1403.0
2020-01-01 13:00:09+00:00,1957.0
...,...
2020-09-30 09:01:54+00:00,1873.0
2020-09-30 11:18:25+00:00,923.0
2020-09-30 14:00:13+00:00,1983.0


In [6]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 839 entries, 15829 to 11825
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   newsdesk    839 non-null    object
 1   section     839 non-null    object
 2   subsection  308 non-null    object
 3   material    839 non-null    object
 4   headline    839 non-null    object
 5   abstract    839 non-null    object
 6   keywords    839 non-null    object
 7   word_count  839 non-null    int64 
 8   pub_date    839 non-null    object
 9   n_comments  839 non-null    int64 
 10  uniqueID    839 non-null    object
dtypes: int64(2), object(9)
memory usage: 78.7+ KB


In [7]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49865 entries, 2893132 to 463789
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   commentID              49865 non-null  int64  
 1   status                 49865 non-null  object 
 2   commentSequence        49865 non-null  int64  
 3   userID                 49865 non-null  int64  
 4   userDisplayName        49849 non-null  object 
 5   userLocation           49845 non-null  object 
 6   userTitle              42 non-null     object 
 7   commentBody            49865 non-null  object 
 8   createDate             49865 non-null  object 
 9   updateDate             49865 non-null  object 
 10  approveDate            49865 non-null  object 
 11  recommendations        49865 non-null  int64  
 12  replyCount             49865 non-null  int64  
 13  editorsSelection       49865 non-null  bool   
 14  parentID               21575 non-null  float64


In [8]:
df_articles.describe(include='O').T

,count,unique,top,freq
newsdesk,839,42,OpEd,89
section,839,32,U.S.,128
subsection,308,37,Politics,77
material,839,8,News,657
headline,839,835,Coronavirus in N.Y.C.: Latest Updates,3
abstract,839,832,Recent residential sales in New York City and ...,3
keywords,839,780,[],34
pub_date,839,834,2020-02-25 10:00:22+00:00,2
uniqueID,839,839,nyt://article/95f224dd-f15f-5f98-ba28-159c3107...,1


In [65]:
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer

# import numpy as np

# vectorizer = CountVectorizer()
# newsgroups_data = fetch_20newsgroups()
# print(newsgroups_data.data[500])
# print()

In [9]:
df_comments.describe(include='O').T

,count,unique,top,freq
status,49865,1,approved,49865
userDisplayName,49849,20041,John,267
userLocation,49845,8257,NYC,1680
userTitle,42,12,The New York Times,14
commentBody,49865,49849,Thank you.,4
createDate,49865,49803,2020-02-16 16:51:10,2
updateDate,49865,49793,2020-10-28 15:09:11,2
approveDate,49865,49237,2020-04-09 15:39:56,3
parentUserDisplayName,21566,10037,Doug,147
commentType,49865,3,comment,28289


# Clean escapes

In [39]:
import nlp_tools
df_comments['commentClean'] = df_comments['commentBody'].apply(nlp_tools.clean)

# Tokenize / Split words

In [40]:
# df_comments['commentTokens'] = df_comments['commentClean'].apply(nlp_tools.tokenizeText)
# df_comments.head(1)

# Lemmatize Tokens

In [41]:
df_comments['commentLemmas'] = df_comments['commentTokens'].apply(nlp_tools.lemmatizeTokenList)
# print(df_comments.head(1))

# Remove Stop Words

In [37]:
df_comments['commentNoStopWords'] = df_comments['commentLemmas'].apply(nlp_tools.cleanStopWords)
# print(df_comments['commentNoStopWords'].head(1))

In [43]:
df_comments[['commentClean','commentTokens','commentLemmas','commentNoStopWords']].head(5)

,commentClean,commentTokens,commentLemmas,commentNoStopWords
2893132,@Jo Williams Exactly I see a nightmarish Gotha...,"[@Jo, Williams, Exactly, I, see, a, nightmaris...","[@jo, williams, exactly, i, see, a, nightmaris...","[@jo, williams, exactly, see, nightmarish, got..."
2599853,@Bugmon Are you really that obtuse? I do wear ...,"[@Bugmon, Are, you, really, that, obtuse, ?, I...","[@bugmon, be, you, really, that, obtuse, ?, i,...","[@bugmon, really, obtuse, ?, wear, mask, go, ...."
125068,It's so sad to see Trump's impact on our natio...,"[It, 's, so, sad, to, see, Trump, 's, impact, ...","[it, 's, so, sad, to, see, trump, 's, impact, ...","['s, sad, see, trump, 's, impact, nation, ., w..."
2425848,The children’s hospital in my city has enormou...,"[The, children, ’s, hospital, in, my, city, ha...","[the, children, ’s, hospital, in, my, city, ha...","[children, ’s, hospital, city, enormous, cash,..."
1444275,I can't wait to watch this virtually with othe...,"[I, can, 't, wait, to, watch, this, virtually,...","[i, can, 't, wait, to, watch, this, virtually,...","['t, wait, watch, virtually, folks, ., grandmo..."


# Count Vectorization

In [51]:
df_comments['commentNoStopWords']

2893132    [@jo, williams, exactly, see, nightmarish, got...
2599853    [@bugmon, really, obtuse, ?, wear, mask, go, ....
125068     ['s, sad, see, trump, 's, impact, nation, ., w...
2425848    [children, ’s, hospital, city, enormous, cash,...
1444275    ['t, wait, watch, virtually, folks, ., grandmo...
                                 ...                        
3009694           [best, last, sentence, column, ,, ever, .]
592853     [roger, stone, ,, along, paul, manafort, lee, ...
1359418    [one, individual, tweet, 798, time, day, ., ’s...
4886496          [@anthony, live, journalists, civilians, ?]
463789     [believe, senator, klobuchar, ’s, act, share, ...
Name: commentNoStopWords, Length: 49865, dtype: object

In [61]:
text = nlp_tools.createCorpus(df_comments['commentNoStopWords'])
len(text)

2179866

In [63]:
len(set(text))

76615

In [57]:
df = nlp_tools.countVectorizer(df_comments['commentNoStopWords'].iloc[0])

    0   1   2   3   4   5   6   7   8   9   10  11  12
0    0   0   0   0   0   1   0   0   0   0   0   0   0
1    0   0   0   0   0   0   0   0   0   0   0   0   1
2    0   0   1   0   0   0   0   0   0   0   0   0   0
3    0   0   0   0   0   0   0   0   1   0   0   0   0
4    0   0   0   0   0   0   1   0   0   0   0   0   0
5    0   0   0   1   0   0   0   0   0   0   0   0   0
6    0   1   0   0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   0   0   1   0
8    1   0   0   0   0   0   0   0   0   0   0   0   0
9    0   0   0   0   0   0   0   0   0   1   0   0   0
10   0   0   0   0   1   0   0   0   0   0   0   0   0
11   0   0   0   0   0   0   0   1   0   0   0   0   0
12   0   0   0   0   0   0   0   0   0   0   1   0   0
13   0   0   0   0   0   0   0   0   0   0   0   0   0


In [58]:
print(df)

    batman  city  exactly  gotham  help  jo  nightmarish  save  see  \
0        0     0        0       0     0   1            0     0    0   
1        0     0        0       0     0   0            0     0    0   
2        0     0        1       0     0   0            0     0    0   
3        0     0        0       0     0   0            0     0    1   
4        0     0        0       0     0   0            1     0    0   
5        0     0        0       1     0   0            0     0    0   
6        0     1        0       0     0   0            0     0    0   
7        0     0        0       0     0   0            0     0    0   
8        1     0        0       0     0   0            0     0    0   
9        0     0        0       0     0   0            0     0    0   
10       0     0        0       0     1   0            0     0    0   
11       0     0        0       0     0   0            0     1    0   
12       0     0        0       0     0   0            0     0    0   
13    

# Pipeline
Process text functions we created with wrapper function

In [34]:
# Pipeline here, call functions in package
pipeline = [str.lower, remove_stopwords]

def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

# Use something like this to process
# df['tokens'] = df['text'].apply(prepare, pipeline=pipeline)

In [24]:
from collections import Counter
tokens = tokenize('This is a test of what is possible with ths tester')
counter = Counter(tokens)

print(counter)

NameError: name 'tokenize' is not defined

## Clean non-essential characters (Remove Noise)

In [4]:
#https://learning.oreilly.com/library/view/blueprints-for-text/9781492074076/ch04.html#ch04removenoiseregex

RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

def impurity(text, min_len=15):
    """returns the ratio of suspicious characters in a text"""
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

### Try out clean and impurity
Want to find noise and the percentage of it

In [7]:
pre_impurity = []
post_impurity = []
clean_comments = []

# Step through each of text comments
for _ in range(0,len(df_comments_sample)):
    # Step through the text comment body row by row
    text = df_comments_sample['commentBody'].iloc[_]
    pre = nlp_tools.impurity(text) # Call impurity to find non-text characters
    clean_text = nlp_tools.clean(text) # Clean up text
    post = nlp_tools.impurity(clean_text) # Check Post impurity

    pre_impurity.append(pre) # Build list of impurity
    post_impurity.append(post) # Build list of post impurity

    clean_comments.append(clean_text) # Make list of cleaned text

max_val = max(post_impurity) # Find max value
ind_max = post_impurity.index(max_val) # find what index max value existed at

print('Max Impurity Val: ' + str(max_val)) # Report max impurity
print('Index Max:' + str(ind_max))

print(df_comments['commentBody'].iloc[ind_max])
print('\n'*2)
print(clean_comments[ind_max])



Max Impurity Val: 0.10526315789473684
Index Max:2673
#IBelieveHer #MeToo



#IBelieveHer #MeToo


### TODO Add new columns

# Character Normalization

In [8]:
# Pick normalization of long comment
len(df_comments_sample['commentBody'].iloc[800])

1001

In [9]:
# Show that comment
df_comments_sample['commentBody'].iloc[800]

'As a Foreign Officer I was proud to represent my country overseas. During my 60+ years associated with Eisenhower Fellowships, which has brought over 2,000 outstanding Fellows to the United States, I was proud to see them observe our American democracy. I often would chide my British relatives for not appreciating  the ‘American way.’\n\nAfter four years of Trump, I have been handed a sharp comeuppance. We are the laughing stock of the world. Our former traditional allies are dismayed by Trump’s abandonment of core principles of common self interest. Authoritarians such as Putin, Kim, Erdogan, and Mohamed bin Salman are laughing, while they play Trump like an accordion.\n\nI recall FDR’s Four Freedoms in 1941. We need Biden reaffirm these freedoms and to stabilize Trump’s shaky ship of state. We must return the United States to the concert of nations that share what had been our decency, our principles, and our alacrity to work with others to protect the oppressed and punish the oppre

In [16]:
text = clean(df_comments_sample['commentBody'].iloc[800])

# text = """The café “Saint-Raphaël” is loca-\nted on Côte dʼAzur."""
# len(text)

In [17]:
# https://learning.oreilly.com/library/view/blueprints-for-text/9781492074076/ch04.html#idm46749279547768
def normalize(text):
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text

In [20]:
text

'As a Foreign Officer I was proud to represent my country overseas. During my 60+ years associated with Eisenhower Fellowships, which has brought over 2,000 outstanding Fellows to the United States, I was proud to see them observe our American democracy. I often would chide my British relatives for not appreciating the ‘American way.’ After four years of Trump, I have been handed a sharp comeuppance. We are the laughing stock of the world. Our former traditional allies are dismayed by Trump’s abandonment of core principles of common self interest. Authoritarians such as Putin, Kim, Erdogan, and Mohamed bin Salman are laughing, while they play Trump like an accordion. I recall FDR’s Four Freedoms in 1941. We need Biden reaffirm these freedoms and to stabilize Trump’s shaky ship of state. We must return the United States to the concert of nations that share what had been our decency, our principles, and our alacrity to work with others to protect the oppressed and punish the oppressors.'

In [18]:
print(normalize(text))

As a Foreign Officer I was proud to represent my country overseas. During my 60+ years associated with Eisenhower Fellowships, which has brought over 2,000 outstanding Fellows to the United States, I was proud to see them observe our American democracy. I often would chide my British relatives for not appreciating the 'American way.' After four years of Trump, I have been handed a sharp comeuppance. We are the laughing stock of the world. Our former traditional allies are dismayed by Trump's abandonment of core principles of common self interest. Authoritarians such as Putin, Kim, Erdogan, and Mohamed bin Salman are laughing, while they play Trump like an accordion. I recall FDR's Four Freedoms in 1941. We need Biden reaffirm these freedoms and to stabilize Trump's shaky ship of state. We must return the United States to the concert of nations that share what had been our decency, our principles, and our alacrity to work with others to protect the oppressed and punish the oppressors.


In [19]:
len(normalize(clean_text))

832

In [15]:
# Use normalization
normalized_col = df_comments_sample['commentBody'].map(normalize)

# Data Masking

### Remove Identifiers like URLs, email, or phone numbers

In [ ]:
from textacy.preprocessing.resources import RE_URL

count_words

In [41]:





tokenizer = RegexpTokenizer(r'\w+')

# Tokenization
tokens = tokenizer.tokenize(normalize(text))
print(tokens)
print(len(tokens))
# count_words(df_comments_sample, column='commentBody', preprocess=RE_URL.findall).head(3)

['The', 'cafe', 'Saint', 'Raphael', 'is', 'located', 'on', 'Cote', 'd', 'Azur']
10
